In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('logs/generated_formula_anchor_node=3_full.csv')

In [4]:
from collections import Counter

In [5]:
def count_entities(formula):
    return Counter(formula)['e']

def count_projections(formula):
    return Counter(formula)['p']

In [6]:
df['original_e_count'] = df.original.map(count_entities)

In [7]:
df

,original_depth,original,DeMorgan,DeMorgan_MultiI,DNF,diff,DNF+diff,DNF+MultiIU,DNF+MultiIUD,formula_id,num_anchor_nodes,original_e_count
0,1,"(i,(n,(p,(e))),(p,(e)))","(i,(n,(p,(e))),(p,(e)))","(i,(n,(p,(e))),(p,(e)))","(i,(n,(p,(e))),(p,(e)))","(d,(p,(e)),(p,(e)))","(d,(p,(e)),(p,(e)))","(i,(n,(p,(e))),(p,(e)))","(D,(p,(e)),(p,(e)))",type0017,2,2
1,2,"(i,(n,(p,(e))),(p,(p,(e))))","(i,(n,(p,(e))),(p,(p,(e))))","(i,(n,(p,(e))),(p,(p,(e))))","(i,(n,(p,(e))),(p,(p,(e))))","(d,(p,(p,(e))),(p,(e)))","(d,(p,(p,(e))),(p,(e)))","(i,(n,(p,(e))),(p,(p,(e))))","(D,(p,(p,(e))),(p,(e)))",type0021,2,2
2,3,"(i,(n,(p,(e))),(p,(p,(p,(e)))))","(i,(n,(p,(e))),(p,(p,(p,(e)))))","(i,(n,(p,(e))),(p,(p,(p,(e)))))","(i,(n,(p,(e))),(p,(p,(p,(e)))))","(d,(p,(p,(p,(e)))),(p,(e)))","(d,(p,(p,(p,(e)))),(p,(e)))","(i,(n,(p,(e))),(p,(p,(p,(e)))))","(D,(p,(p,(p,(e)))),(p,(e)))",type0028,2,2
3,1,"(i,(i,(n,(p,(e))),(p,(e))),(p,(e)))","(i,(i,(n,(p,(e))),(p,(e))),(p,(e)))","(I,(n,(p,(e))),(p,(e)),(p,(e)))","(i,(i,(n,(p,(e))),(p,(e))),(p,(e)))","(i,(d,(p,(e)),(p,(e))),(p,(e)))","(i,(d,(p,(e)),(p,(e))),(p,(e)))","(I,(n,(p,(e))),(p,(e)),(p,(e)))","(D,(i,(p,(e)),(p,(e))),(p,(e)))",type0142,3,3
4,2,"(i,(i,(n,(p,(e))),(p,(p,(e)))),(p,(e)))","(i,(i,(n,(p,(e))),(p,(p,(e)))),(p,(e)))","(I,(n,(p,(e))),(p,(e)),(p,(p,(e))))","(i,(i,(n,(p,(e))),(p,(p,(e)))),(p,(e)))","(i,(d,(p,(p,(e))),(p,(e))),(p,(e)))","(i,(d,(p,(p,(e))),(p,(e))),(p,(e)))","(I,(n,(p,(e))),(p,(e)),(p,(p,(e))))","(D,(i,(p,(e)),(p,(p,(e)))),(p,(e)))",type0148,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
296,3,"(u,(p,(e)),(u,(p,(p,(e))),(p,(p,(p,(e))))))","(n,(i,(n,(n,(i,(n,(p,(p,(e)))),(n,(p,(p,(p,(e)...","(n,(i,(n,(n,(i,(n,(p,(p,(e)))),(n,(p,(p,(p,(e)...","(u,(p,(e)),(u,(p,(p,(e))),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(p,(e))),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(p,(e))),(p,(p,(p,(e))))))","(U,(p,(e)),(p,(p,(e))),(p,(p,(p,(e)))))","(U,(p,(e)),(p,(p,(e))),(p,(p,(p,(e)))))",type0456,3,3
297,2,"(u,(p,(e)),(u,(p,(e)),(p,(p,(e)))))","(n,(i,(n,(n,(i,(n,(p,(e))),(n,(p,(p,(e))))))),...","(n,(i,(n,(n,(i,(n,(p,(e))),(n,(p,(p,(e))))))),...","(u,(p,(e)),(u,(p,(e)),(p,(p,(e)))))","(u,(p,(e)),(u,(p,(e)),(p,(p,(e)))))","(u,(p,(e)),(u,(p,(e)),(p,(p,(e)))))","(U,(p,(e)),(p,(e)),(p,(p,(e))))","(U,(p,(e)),(p,(e)),(p,(p,(e))))",type0457,3,3
298,3,"(u,(p,(e)),(u,(p,(p,(p,(e)))),(p,(p,(p,(e))))))","(n,(i,(n,(n,(i,(n,(p,(p,(p,(e))))),(n,(p,(p,(p...","(n,(i,(n,(n,(i,(n,(p,(p,(p,(e))))),(n,(p,(p,(p...","(u,(p,(e)),(u,(p,(p,(p,(e)))),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(p,(p,(e)))),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(p,(p,(e)))),(p,(p,(p,(e))))))","(U,(p,(e)),(p,(p,(p,(e)))),(p,(p,(p,(e)))))","(U,(p,(e)),(p,(p,(p,(e)))),(p,(p,(p,(e)))))",type0459,3,3
299,3,"(u,(p,(e)),(u,(p,(e)),(p,(p,(p,(e))))))","(n,(i,(n,(n,(i,(n,(p,(e))),(n,(p,(p,(p,(e)))))...","(n,(i,(n,(n,(i,(n,(p,(e))),(n,(p,(p,(p,(e)))))...","(u,(p,(e)),(u,(p,(e)),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(e)),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(e)),(p,(p,(p,(e))))))","(U,(p,(e)),(p,(e)),(p,(p,(p,(e)))))","(U,(p,(e)),(p,(e)),(p,(p,(p,(e)))))",type0460,3,3


In [8]:
from fol.foq_v2 import parse_formula

In [9]:
def count_query_depth(query):
    if query.__o__ == 'e':
        return 0
    elif query.__o__ in 'uiUID':
        return max(count_query_depth(q) for q in query.sub_queries)
    elif query.__o__ == 'p':
        return count_query_depth(query.query) + 1
    elif query.__o__ == 'n':
        return count_query_depth(query.query)
    elif query.__o__ == 'd':
        return max(count_query_depth(query.lquery), count_query_depth(query.rquery))
    else:
        raise NotImplementedError

In [10]:
def count_depth(formula):
    try:
        query = parse_formula(formula)
    except:
        print(formula)
    return count_query_depth(query)

In [11]:
forms = ['original', 'DeMorgan', 'DNF', 'diff', 'DNF+diff', 'DNF+MultiIU', 'DNF+MultiIUD']

In [12]:
for f in forms:
    df[f"{f}:e_counts"] = df[f].map(count_entities)
    df[f"{f}:p_counts"] = df[f].map(count_projections)
    df[f"{f}:p_depth"] = df[f].map(count_depth)

In [13]:
parse_formula("(p,(D,(p,(e)),(p,(e)),(p,(p,(e)))))")

In [14]:
from collections import defaultdict
show_data = defaultdict(list)

In [15]:
for f in forms:
    show_data['form'].append(f)
    show_data['e_counts-mean'].append(df[f"{f}:e_counts"].mean())
    show_data['e_counts-std'].append(df[f"{f}:e_counts"].std())
    show_data['p_counts-mean'].append(df[f"{f}:p_counts"].mean())
    show_data['p_counts-std'].append(df[f"{f}:p_counts"].std())
    show_data['p_depth-mean'].append(df[f"{f}:p_depth"].mean())
    show_data['p_depth-std'].append(df[f"{f}:p_depth"].mean())
    

In [16]:
statdf = pd.DataFrame(show_data)

In [17]:
statdf

,form,e_counts-mean,e_counts-std,p_counts-mean,p_counts-std,p_depth-mean,p_depth-std
0,original,2.893688,0.339598,5.395349,1.416514,2.554817,2.554817
1,DeMorgan,2.893688,0.339598,5.395349,1.416514,2.554817,2.554817
2,DNF,3.122924,0.584385,5.996678,1.841192,2.554817,2.554817
3,diff,2.893688,0.339598,5.395349,1.416514,2.554817,2.554817
4,DNF+diff,3.122924,0.584385,5.996678,1.841192,2.554817,2.554817
5,DNF+MultiIU,3.122924,0.584385,5.996678,1.841192,2.554817,2.554817
6,DNF+MultiIUD,3.122924,0.584385,5.996678,1.841192,2.554817,2.554817


group by the queries by depth and number of queres

In [18]:
for (ec, pc), subdf in df.groupby(["original:e_counts", "original:p_depth"]):
    print(ec, pc, len(subdf[forms]))

1 1 1
1 2 1
1 3 1
2 1 3
2 2 10
2 3 13
3 1 12
3 2 91
3 3 169


In [19]:
for (ec, pc), subdf in df.groupby(["DNF:e_counts", "DNF:p_depth"]):
    print(ec, pc, len(subdf[forms]))

1 1 1
1 2 1
1 3 1
2 1 3
2 2 10
2 3 13
3 1 8
3 2 67
3 3 128
4 1 4
4 2 24
4 3 41
